Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql

In [ ]:
yelp_df["loc_rating"] *= 2 # Converting an /5 rating to /10, this is probably inaccurate

all_poi_df = pd.concat([foursquare_df, yelp_df])
all_poi_df

In [ ]:
def count_unique_values(column):
    return column.nunique()

grouped_all_poi_df = all_poi_df.groupby("station_name", as_index=False).agg({"station_distance_(m)": np.mean, "loc_address": count_unique_values, "loc_category": count_unique_values, "loc_rating": np.mean})
grouped_all_poi_df = grouped_all_poi_df.rename(columns={"station_distance_(m)": "avg_distance_(m)", "loc_address": "num_locations", "loc_category": "loc_types", "loc_rating": "avg_rating"})
grouped_all_poi_df

In [ ]:
all_data_df = pd.merge(stations_df, grouped_all_poi_df, on="station_name")
all_data_df

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [ ]:
# Histrogram to see distribution of number of available bikes across all bike stations
plt.hist(all_data_df["free_bikes"], bins=35, color="green")
plt.xlabel("Available Bikes")
plt.ylabel("Stations")
sns.set_style("dark")
plt.show()

In [ ]:
# Scatterplot measuring available bikes based on average distance of POIs
plt.scatter(all_data_df["avg_distance_(m)"], all_data_df["free_bikes"])
plt.xlabel('Average Distance from Station(m)')
plt.ylabel('Available Bikes')
plt.xticks(fontsize=9)
plt.yticks(fontsize=9)
plt.title("Free Bikes based on Distance")
sns.set_theme()
sns.set_style("dark")

plt.show()

In [ ]:
# Scatterplot measuring available bikes based on average rating of POIs
plt.scatter(all_data_df["avg_rating"], all_data_df["free_bikes"])
plt.xlabel('Average Rating')
plt.ylabel('Available Bikes')
plt.title("Free Bikes based on Rating")
plt.xticks(fontsize=9)
plt.yticks(fontsize=9)
sns.set_theme()
sns.set_style("dark")

plt.show()

In [ ]:
'''

Initial viewing of the visualized data doesn't show an immediately obvious pattern. Looking at the histogram, while there is not a strong curve yet in the visual, the shape of one is starting to appear.
A bike station is most likely to have 12 available bikes at a time. By comparing that information to the scatterplots, it seems there is a relationship between available bikes and distance, where stations with closer POIs have less
available bikes (<12), perhaps since they are being used more. Similarly, the relationship between available bikes and distance seems to show that there are less available bikes at stations where many POIs have a high rating, also perhaps
since they are being used by people attending those places. The visualizations could be improved with a larger data sample.

'''

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [ ]:
path = "../data/"
db = path + "poi.db"
conn = sql.connect(db)
cursor = conn.cursor()

foursquare_df.to_sql("foursquare_poi", conn, index=False, if_exists="replace")
yelp_df.to_sql("yelp_poi", conn, index=False, if_exists="replace")
all_poi_df.to_sql("all_poi", conn, index=False, if_exists="replace")
all_data_df.to_sql("all_data", conn, index=False, if_exists="replace")
conn.commit()

Look at the data before and after the join to validate your data.

In [ ]:
validation = '''
WITH pre_join AS(
SELECT * FROM foursquare_poi 
UNION 
SELECT * FROM yelp_poi
)

SELECT *
FROM all_poi,
     pre_join
WHERE all_poi.loc_name != pre_join.loc_name
'''

cursor.execute(validation)
validation_results = cursor.fetchall()

for result in validation_results:
    print(result)

cursor.close()
conn.close()